In [22]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize)

In [23]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
(LinearDiscriminantAnalysis as LDA,
QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [25]:
Smarket = load_data('Smarket')
train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
Smarket_test.shape

(252, 9)

In [28]:
X_train, X_test = X.loc[train], X.loc[~train]
y_train, y_test = y.loc[train], y.loc[~train]
glm_train = sm.GLM(y_train,
X_train,
family=sm.families.Binomial())
results = glm_train.fit()
probs = results.predict(exog=X_test)
print(probs)

998     0.528220
999     0.515669
1000    0.522652
1001    0.513854
1002    0.498334
          ...   
1245    0.483637
1246    0.506048
1247    0.516658
1248    0.516124
1249    0.508072
Length: 252, dtype: float64


In [33]:
labels = np.array(['Down']*len(probs))
labels[probs >0.5] = "Up"

In [34]:
D = Smarket.Direction
L_train, L_test = D.loc[train], D.loc[~train]

In [41]:

ct =confusion_table(labels, L_test)
print(ct)

Truth      Down  Up
Predicted          
Down         77  97
Up           34  44


In [56]:
TN = ct.iloc[0,0]
TP = ct.iloc[1,1]
FN = ct.iloc[0,1]
FP = ct.iloc[1,0]
Accuracy = (TP + TN) /(TN+TP+FN+FP)
Recall = TP /(TP+FN)
Precision = TP/(TP+FP)
F_score = Recall*Precision/(Recall +Precision)
df_r = pd.DataFrame(
 {'Accuracy': [Accuracy]
  ,  'Precision':[Precision],   'Recall':[Recall],    'F_score':[F_score]
 }
)
df_r

,Accuracy,Precision,Recall,F_score
0,0.480159,0.564103,0.312057,0.200913


In [60]:
from siuba import *

In [86]:
dt_1 =(
    X_train >>
    mutate(y=y_train)>>
    mutate(y=if_else(y,1,0))
    
)
data.columns
dt_1.head()

,intercept,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,y
0,1.0,0.381,-0.192,-2.624,-1.055,5.010,1.1913,1
1,1.0,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1
2,1.0,1.032,0.959,0.381,-0.192,-2.624,1.4112,0
3,1.0,-0.623,1.032,0.959,0.381,-0.192,1.2760,1
4,1.0,0.614,-0.623,1.032,0.959,0.381,1.2057,1


In [152]:
formula= "y ~ intercept + Volume + Lag1 + Lag2 + Lag3 + Lag4 + Lag5"
formula= "y ~ intercept + Volume + Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + I(intercept**2) + I(Volume**2)\
+I(Lag1**2) +I(Lag2**2)+I(Lag3**2)+I(Lag4**2)+I(Lag5**2)"
formula= "y ~ Lag1 + Lag2 + I(Lag1**2)*I(Lag2**2) + I(Lag1**3)*I(Lag2**3)"

In [153]:
import statsmodels.formula.api as smf
glm_train = smf.glm(formula=formula, data=dt_1,family=sm.families.Binomial())
mod2=glm_train.fit()
print(mod2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  998
Model:                            GLM   Df Residuals:                      989
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -685.23
Date:                Mon, 06 Nov 2023   Deviance:                       1370.5
Time:                        15:29:50   Pearson chi2:                     995.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01275
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [154]:
probs = mod2.predict(exog=X_test)
# print(probs)
labels = np.array(['Down']*len(probs))
labels[probs >0.485] = "Up"
D = Smarket.Direction
L_train, L_test = D.loc[train], D.loc[~train]
ct =confusion_table(labels, L_test)
print(ct)
TN = ct.iloc[0,0]
TP = ct.iloc[1,1]
FN = ct.iloc[0,1]
FP = ct.iloc[1,0]
Accuracy = (TP + TN) /(TN+TP+FN+FP)
Recall = TP /(TP+FN)
Precision = TP/(TP+FP)
F_score = Recall*Precision/(Recall +Precision)
df_r = pd.DataFrame(
 {'Accuracy': [Accuracy]
  ,  'Precision':[Precision],   'Recall':[Recall],    'F_score':[F_score]
 }
)
df_r

Truth      Down   Up
Predicted           
Down         26   18
Up           85  123


,Accuracy,Precision,Recall,F_score
0,0.59127,0.591346,0.87234,0.352436
